<hr>

# PREDICTING THE STOCK MARKET WITH WATSON

## Part I: Introduction

In this Jupyter Notebook you will learn step-by-step how to extract financial data from one of the most popular public databases for econometric data, as well as the cleansing and preparation processes of the data science traditional workflow.

After preparing the data into an adequate format, structured into a pandas dataframe, we train a predictive model utilizing techniques such as curve-fitting and ARIMA (AutoRegressive Integrated Moving Average).

Lastly, we create interactive plots to visualize the collected data, as well as the results of the forecasters trained.

## Table of Contents

#### 1. Mining Stock Market Data
* 1.1. Quandl API Setup
* 1.2. Downloading the WIKI/PRICES Table
* 1.3. Visualizing the Collected Data

#### 2. Training a Predictive Model using Open-Source Libraries
* 2.1. Fbprophet Setup
* 2.2. Data Preparation
* 2.3. The Machine Learning

####  3. Analyzing the Results
* 3.1. Analyzing Seasonalities
* 3.2. Overlapping of Actual Values with Expected Values
* 3.3. Trained Model Evaluation

#### 4. Exporting Data
* 4.1. Exporting Data to IBM Cloud Object Storage

<hr>

# 1: Mining Stock Market Data

The data mining step is done using the Quandl API, a financial database (with both public and private tables) purchased by NASDAQ in 2018. It is not mandatory to use an API key to extract data, however it is recommended to generate one for free <a href="https://www.quandl.com/">at the Quandl website</a>.

### 1.1: Quandl API Setup

In [ ]:
# Install quandl package (this can take some minutes)
!pip install --user quandl==3.4.8 --upgrade

In [ ]:
# Import quandl package
import quandl
print("Quandl package imported.")

If you registered and generated an API key for free on the Quandl web page, type it in the variable in the cell below.

In [ ]:
quandl.ApiConfig.api_key = ""

### 1.2: Downloading the WIKI/PRICES Table

Quandl has several public and private data tables available.

In this project we will use the table <a href="https://www.quandl.com/databases/WIKIP"> WIKI/PRICES</a>. This table has information on the daily closing and opening values of the shares of more than 3000 companies.

The data download is done via an API call executed with the quandl library, using the `get_table` function. The arguments of this function are: the table name, the <a href="https://en.wikipedia.org/wiki/Ticker_symbol"> ticker symbol </a> of the desired stock, and the desired columns of the table. The `paginate=True` parameter allows more than 10000 rows of data to be transferred with a single call.

The result of this function call is a pandas dataframe with the desired stock data, ready to be transformed.

In [ ]:
# API Call example - Downloading financial data from IBM stocks
data = quandl.get_table('WIKI/PRICES',
                        ticker='IBM', 
                        qopts={'columns':['date', 'open', 'high', 'low', 'close']},
                        paginate=True)

At the <a href="https://www.nasdaq.com/symbol/">NASDAQ page</a> you can see the ticker symbols for various stocks.

You can also choose a ticker from the ones listed in the `nasdaq-ticker-table.csv` file, which can be accessed <a href ="https://github.com/vanderleipf/ibmdegla-ws-projects/blob/master/EN-US/forecasting-the-stock-market/data/stock-ticker-table.csv">here</a>.

Choose the ticker of another company, or use the data already configured and saved in the dataframe `date` by the API call executed in the above code cell.

In [ ]:
# Renaming the pandas dataframe columns
data.columns = ['Date', 'Open', 'High', 'Low', 'Close']

# Reset the pandas dataframe index
data = data.reset_index(drop=True)

# Show the last five rows of the pandas dataframe
data.tail()

### 1.3: Visualizing the Collected Data

To view the data, the `bokeh` package is used. This module is capable of generating interactive Javascript charts.

In [ ]:
!pip install --user bokeh==1.0.4 --upgrade

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.io import output_notebook

print('Packages imported.')

In [ ]:
# Load bokeh
output_notebook()

In [ ]:
# Bokeh Figure
p = figure(plot_width=1200, plot_height=550, title='Stock Value Historical Data', x_axis_type="datetime")

# Plot Lines
p.line(data.Date, data.Open, line_width=2, line_color="#0099ff", legend='Open')
p.line(data.Date, data.Close, line_width=2, line_color="#ff6699", legend='Close')
p.line(data.Date, data.High, line_width=1, line_color="#000000", legend='High')
p.line(data.Date, data.Low, line_width=1, line_color="#000000", legend='Low')

# Axis and Labels
p.legend.orientation = "vertical"
p.xaxis.axis_label = "Date"
p.xaxis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = "Value ($ USD)"
p.yaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '12pt'

In the figure below we can see and interact with the data extracted from the WIKI/PRICES table. In blue we have the opening value of the stock; in red the closing value of the stock; in black we have the maximum and minimum values (daily).

In [ ]:
show(p)

<hr>

# 2: Training a Predictive Model using Open-Source Libraries

In the training process of our predictive model, we will use the `fbprophet` package, developed by Facebook for time series analysis.

Fbprophet follows the same style of objects as `sklearn`, an extremely popular machine learning python library. An instance of the Prophet class is created and then the `fit` and` predict` methods are used.

The training set of a Prophet model is a two-column pandas dataframe with the columns `ds` and` y`. The `ds` (datestamp) column must be a date in the format YYYY-MM-DD, or a timestamp in the format YYYY-MM-DD HH:MM:SS. The `y` column must be numeric, and represents the variable we wish to model in the future.

### 2.1: Fbprophet Setup

In [ ]:
# Install required packages (this can take some minutes)
# UPDATE(vnderlev): setuptools-git is now required for fbprophet 0.5. 
!pip install --user pystan==2.17.1.0 holidays==0.9.8 setuptools-git==1.2 matplotlib==3.0.2 --upgrade

In [ ]:
# Install fbprophet from conda-forge (this will take some time, please be patient)
!conda install -c conda-forge fbprophet=0.5 -y

In [ ]:
# Import packages
import fbprophet
import datetime as dt

print('Packages imported.')

### 2.2: Data Preparation

First of all, the previously collected dataframe needs to be prepared.

The preparation includes the selection of a subset of the data for model training. In this Jupyter notebook we will only sample the latest data, starting from 2008.

Of the four columns with stock values, only one column will be chosen as the variable of interest. We will choose the daily opening value of the shares (the `Open` column).

In [ ]:
# Select train sample
df_train = data.copy()
df_train = df_train[(df_train.Date > dt.datetime(2008,1,1))]
df_train.tail()

In [ ]:
# Format the dataframe for FBProphet
df_train.rename(columns={'Open':'y', 'Date':'ds'}, inplace=True)
df_train = df_train.filter(items=['ds', 'y'])
df_train.tail()

### 2.3: The Machine Learning

As previously mentioned, Prophet uses the same interface style as the sklearn library.

A Prophet class is instantiated, with the desired types of seasonality. For this work, we will consider annual and monthly seasonalities in stock values, and disregard daily and weekly effects.

Usually the presented settings are the most adequate for stock prediction, but feel free to test different seasonalities.

In [ ]:
# Instantiate a fbprophet model
model = fbprophet.Prophet(daily_seasonality=False,
                weekly_seasonality=False, 
                yearly_seasonality=True,
                changepoint_prior_scale=0.05,
                changepoints=None)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

In [ ]:
# Train model
model.fit(df_train)

In the following cell, the prediction runs in a period of 365 days (1 year) in the future. The result is a multi-column dataframe with information about trends, tolerances, and forecasts.

In [ ]:
# Execute forecasting algorithm (1 year into the future)
future_data = model.make_future_dataframe(periods=365, freq='D')
future_data = model.predict(future_data)
future_data.tail()

<hr>

## 3: Results

### 3.1: Analyzing Seasonalities

]In the figure below we have a quick plot of the results using the Prophet class `plot` method.

The black dots are the actual data, the dark blue line is the modeled trend, and in light blue we have the tolerance (minimum and maximum values modeled).

In [ ]:
fig1 = model.plot(future_data)

Using the `plot_components` function of the Prophet object, we can also analyze trend information, as well as the monthly and yearly seasonalities considered in the modeling.

In [ ]:
fig2 = model.plot_components(future_data)

### 3.2: Overlapping of Actual Values with Expected Values

In [ ]:
# Bokeh Figure
p = figure(plot_width=1200, plot_height=550, title='Stock Value Historical Data', x_axis_type="datetime")

# Plot Lines
p.line(data[data['Date'] > dt.datetime(2008,1,1)].Date, data[data['Date'] > dt.datetime(2008,1,1)].Open, line_width=2, line_color="#0099ff", legend='Observed Open Value')
p.line(future_data.ds, future_data.yhat, line_width=2, line_color="#2B0000", legend='Modeled Open Value')
p.line(future_data.ds, future_data.yhat_upper, line_width=0.5, line_color="#000099", legend='Upper Estimates')
p.line(future_data.ds, future_data.yhat_lower, line_width=0.5, line_color="#000099", legend='Lower Estimates')

# Axis and Labels
p.legend.orientation = "vertical"
p.xaxis.axis_label = "Date"
p.xaxis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = "Value ($ USD)"
p.yaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '12pt'

In the next cell we can thoroughly analyze the quality of the trained model by comparing the modeled values (in black) with the actual observed values (light blue).

In [ ]:
show(p)

### 3.3: Trained Model Evaluation

The Prophet class also provides the means to perform an evaluation of the created model.

The `cross-validation` method will be used to evaluate our model.

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon='365 days')
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

Analyzing the `MAPE` criterion projected in the cell above, it is noticed that the error for a forecast of up to 50 days turns around 5%, reaching 15% for a forecast of 365 days. This error is a relatively good metric if the objective is to construct an image of the market trends. For day-trading purposes, even 95% accuracy is not really adequate.

<hr>

# 4: Exporting Data

### 4.1: Exporting Data to IBM Cloud Object Storage

The mined data, as well as the data produced by the model, can be exported as CSV files to IBM Cloud Object Storage, and eventually be used or published in other applications.

Next we present the code necessary to perform this task in an automated way.

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3, os

print('Packages imported.')

Set up your IBM Cloud Object Storage credentials in the next cell.

In [ ]:
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
cos_credentials = {
    'IAM_SERVICE_ID': '',
    'IBM_API_KEY_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
    'FILE': ''
}


The `upload_data_to_ibm_cos` is a function created to upload pandas dataframes as CSV files in IBM COS.

In [ ]:
def upload_data_to_ibm_cos(credentials, df, df_future, tick):
    cos = ibm_boto3.client(service_name='s3', 
                           ibm_api_key_id=credentials['IBM_API_KEY_ID'], 
                           ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                           ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                           config=Config(signature_version='oauth'),
                           endpoint_url=credentials['ENDPOINT'])
    df.to_csv('{}.csv'.format(tick), sep=',', encoding='utf-8')
    df_future.to_csv('{}_future.csv'.format(tick), sep=',', encoding='utf-8')
    try:
        res=cos.upload_file(Filename='{}.csv'.format(tick), Bucket=credentials['BUCKET'], Key='{}.csv'.format(tick))
        res=cos.upload_file(Filename='{}_future.csv'.format(tick), Bucket=credentials['BUCKET'], Key='{}_future.csv'.format(tick))
    except Exception as e:
        print(Exception, e)
    else:
        print("{} data uploaded to IBM COS.".format(tick))
        os.remove('{}.csv'.format(tick))
        os.remove('{}_future.csv'.format(tick))

In [ ]:
# Upload mined and modeled data to IBM COS
upload_data_to_ibm_cos(cos_credentials, data, future_data, 'IBM')

You can now access the `<stock_ticker>.csv` and `<stock_ticker>_future.csv` file anywhere using the IBM COS API.

<hr>


## Want to Learn More?

Watch the public video on how to replicate this analysis with SPSS Modeler, an analytics platform that has several embedded machine learning algorithms. This platform is designed to facilitate data analysis through a graphical programming language, and is integrated with Watson Studio.

This notebook and its source code is made available under the terms of the <a href = "https://github.com/IBM/watson-stock-market-predictor/blob/master/LICENSE">Apache License 2.0</a>.

<hr>

### Thank you for completing this journey!